# Pandas

pandas is a software package written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating tabular data and time series. 


## Learning objective

We will use an example from a real research project and use pandas to walk through steps that social scientists usually take to handle raw data.


## 1. Import pandas

To begin, let's look at a few ways to import pandas

In [ ]:
# import specific classes and functions: useful if you know you'll only be needing the most common parts of pandas
from pandas import DataFrame, read_csv

# General syntax to import a package but no functions: 
#   import (package) as (give the package a nickname/alias)
# Most pandas users will import the entire package and give it the short nickname "pd" as follows:
import pandas as pd

## 2. Data example

We are going to use an example dataset retrieved from Stack Overflow.

Stack Overflow is a question and answer website. Users can ask and answer questions related to programming. Here is an example question and all the answers:

https://stackoverflow.com/questions/4/how-to-convert-a-decimal-to-a-double-in-c

Stack Overflow data is publicly available, which allows researchers to observe user behavior in the community.

Here is an overview of the data schema:

https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

Stack Overflow data is commonly used in social science research, especially studies about expertise and knowledge sharing! Here are just a few examples:
- [How do programmers ask and answer questions on the web?](https://dl.acm.org/doi/abs/10.1145/1985793.1985907)
- [Discovering Value from Community Activity on Focused Question Answering Sites: A Case Study of Stack Overflow](https://dl.acm.org/doi/abs/10.1145/2339530.2339665)

## 3. Import csv data

As mentioned in the introduction, pandas is designed to work with tabular data. Tabluar data is most commonly provided to researchers in a simple spreadsheet format known as csv, which stands for *comma separated values*. Because of this, pandas has a function to load tabular data from a csv file:  `read_csv`. Let us take a look at this function and what inputs it takes.

In [ ]:
help(read_csv)

Common parameters in **read_csv**

read_csv(filepath, sep= , header= , usecols=None)

- filepath_or_buffer : str, path object or file-like object
   
- sep : str, default ','
    
- header : int, list of int
 - Row number(s) to use as the column names, and the start of the data.  
    
- names : array-like, optional
 - List of column names to use, if the file contains a header row
   
- usecols : list-like or callable
 - Return a subset of the columns. 
 - For example, a valid list-like
    `usecols` parameter would be ``[0, 1, 2]`` or ``['foo', 'bar', 'baz']``.
   


As we can see from the parameters description, we need to give `read_csv` a path to the csv file we want to load. Here, we will load the example StackOverflow data, which is contained in the file "so_question.csv" in our GitHub repo.

In [ ]:
# The following path is valid for our Binder setup. If you are running this notebook in your local machine instead and have the file downloaded to a different location, you may need to change this path.
Location = './data/so_question.csv'
df = pd.read_csv(Location)

`read_csv` loads the data into a structure known as a **DataFrame**, as we can see if we check the type of the variable `df` that we created:

In [ ]:
type(df)

Think of a **DataFrame** as a table or spreadsheet that you can work with inside your Python code! Like any table, it contains data organized into rows and columns.

Now you might be wondering what the data actually looks like. The best way to find out is to use the `display` function, which will show a sample of data from the DataFrame. This is always the first thing you should do after loading a new dataset in pandas!

In [ ]:
display(df)

This brings us to the first problem of the exercise. The `read_csv` function treated the first record in the csv file as the header names. This is obviously not correct since the text file did not provide us with header names; instead, we can see that the first row looks like real data.

To correct this we will pass the `header` parameter to the read_csv function and set it to **None** (means null in python).

In [ ]:
df = pd.read_csv(Location, header=None)
display(df)

We can see that all 10000 rows of the csv file are now correctly treated as real data. Since no header names were given, pandas automatically just numbers the headers starting from 0. This is obviously not very useful, usually you want your columns to have meaningful names describing what kind of data they contain. In this case, the documentation for the StackOverflow dataset tells us what the columns are: 
- The first column is an ID
- The second column is a timestamp
- The third column is a topic tag
- The fourth column is a year
- The fifth column is a month
- The sixth column is the ID of the accepted answer, if any
- The seventh column is the ID of the author

Now we want to tell pandas to use column names that are descriptive of this data. We can do this by passing a list of column names to the parameter `names`. This also lets us omit the `header` parameter.

In [ ]:
df = pd.read_csv(Location, names=['id', 'creation_time', 'tag', 'year','month', 'accepted_id','owner_user_id'])
display(df)

Notice the leftmost column is special: it does not have a name and simply contains consecutive numbers [0,1,2,3,4,...]. This is the *index* of the DataFrame; you can think of these as the row numbers in an Excel file. This index is also similar to the primary key of a sql table with the exception that an index is allowed to have duplicates.

One last thing we want to point out before we move on: notice that in the accepted_id column, there seems to be a mix of data: some look like numerical IDs, but others say "NaN". NaN is short for "Not A Number" and is pandas' way of representing *missing data*. Think of NaNs as equivalent to an empty cell in an Excel file. The meaning of missing data depends on the context of the dataset. In this case, since this data is from Stack Overflow and the "accepted_id" column represents the ID of an accepted answer, we can infer that in this case missing data means that the question never received an accepted answer. In research, we often want to do something special to handle missing data; we will return to this later in the workshop.

## 4. Checking the structure of the data
Usually we first do a couple of sanity checks to make sure the data we imported is in good shape and understand its high level structure. 

Usually we make sure the data type of each variable is sensible, check the total number of observations, and the number of variables in the data set.

Let's take a look at the example data set. First, we'll check data types using the `dtypes` variable.


In [ ]:
# Check data type of the columns
df.dtypes

In [ ]:
# Check data type of a specific column
df.accepted_id.dtype

Next we'll check the number of observations (rows) in the dataset. This can be done by examining the length of the DataFrame's *index*:

In [ ]:
len(df.index)

Similarly, we can check the number of variables (columns) in the dataset by examining the length of the DataFrame's *columns* list:

In [ ]:
len(df.columns)

## 5. Slicing and Indexing
Usually, we will not be working with an entire data set all at the same time. Instead, we usually want to pick out specific rows or columns to analyze and work with. In pandas, this can be done using *slicing and indexing*.

The most basic indexing operation is selecting a specific column. This can be done using standard Python indexing syntax (square brackets) and giving it the name of the column you want. For example, the following code will select the "id" column:

In [ ]:
id_column = df['id']
display(id_column)

The syntax for selecting a row is similar, except that you need to tell pandas that you are trying to access a row, since indexing defaults to columns. To do this, use the `loc` variable:

In [ ]:
fifth_row = df.loc[4] # remember the index starts from 0, so the fifth row is at index 4
display(fifth_row)

Both row indexing and column indexing allow you to select multiple rows or columns at once. To do this, you can provide a **list** of row indices or column names, instead of just one:

In [ ]:
multi_columns = df[['id', 'creation_time']]
display(multi_columns)

In [ ]:
multi_rows = df.loc[[0,1,2,3]]
display(multi_rows)

Since row indices are numerical, python's **slicing** syntax can also be used on them to select a range of rows. This can simplify your code a lot because you can specify a range rather than writing out full list of indices! For example, the following code is equivalent to the previous code:

In [ ]:
multi_rows = df.loc[0:3] # important difference from regular python: pandas slices include *both* indices!
display(multi_rows)

Additionally, we may sometimes want to pick out a **single** value from the DataFrame. For example, suppose we want to find the question ID in the fifth row. We can do this by giving both a row and column to the `loc` indexer. Note that the row and column **must** be given in that order (you cannot give the column first) and should be separated by a comma, as shown below:

In [ ]:
fifth_id = df.loc[4,"id"]
print(fifth_id)

## Exercise 1
Finally, the same syntax can be used to select multiple rows **and** columns at the same time; you simply need to provide a list or range of rows, followed by a list of column names, separated by a comma.

Let's try get first 4 rows, for column 'id' and 'creation_time'

In [ ]:
multi_row_and_col = df.loc[____,[_____]]
display(multi_row_and_col)

## Exercise 2

What is the tag of the 50th question in the data set?

In [ ]:
print(df.loc[49,"tag"])

## 6. Summarizing column data
Previously, we sanity checked our data by checking its structure (data types and number of rows and columns). A second common step in preliminary exploration of the data is to examine individual columns (with the help of indexing, as covered previously). pandas can help with this process by offering several methods to *summarize* the data in a column.

### Summarizing numerical data
Some columns contain numerical data. For example, years in the "year" column are represented as numbers. For numerical data, pandas offers a number of methods that implement common mathematical summary functions, such as finding the min and the max:

In [ ]:
print(df["year"].min()) # what's the earliest year in the data?
print(df["year"].max()) # what's the latest year in the data?

### Summarizing categorical data
Mathematical operations like min and max are useful for quickly summarizing numerical data. But not all columns are numerical. For example, the "tag" column contains text. But notice that the text in the "text" column is not just any arbitrary text! Rather, it seems to take on specific, repeated values, like ".htaccess" or "zxing". Therefore, "tag" is what data scientists refer to as a *categorical variable*: one that can only take on values from a fixed, finite set of possibilities. To summarize categorical data, researchers generally want to know what is the set of values the variable can take? In pandas, this can be done using the `unique` method:

In [ ]:
print(df["tag"].unique())

## Exercise 2

- Which years and months of questions are in the data?


In [ ]:
print(____________) # fill in your code for years here
print(____________) # fill in your code for months here

## 7. Querying using conditions based on column values

Now we can start looking at more advanced data analysis. Previously, we selected subsets of rows by giving the `loc` indexer a list or range of row indices. Most often, however, we don't know ahead of time which rows we want to analyze; instead, we more often want to select rows based on some conditions. For example, we might want all questions that took place in a specific year, or all questions that have a specific tag. To achieve this, the `loc` indexer can actually be used with conditions, not just with indices! 

This is best explained through examples. Let's start with the first example question, looking for all questions that took place in a specific year, let's say 2008. The syntax for the comparison would be `df['year'] == 2008`, and we can pass this comparison directly to `loc`, as follows:

In [ ]:
year2008 = df.loc[df['year'] == 2008]
display(year2008)

Notice that this syntax is similar to the filtering syntax used by numpy (as seen in the previous Python workshop) and by R. Alternatively, if you are familiar with SQL, you can think of the above code as being equivalent to the SQL query `select * from df where year=2008`.

Like in numpy, you can include multiple comparisons in a single query, combining them using operators `&` for "and" and `|` for "or". So, for example, if we wanted only questions from August 2008:

In [ ]:
august2008 = df.loc[(df['year']==2008)&(df['month']==8)]
display(august2008)

Notice that when we run a query, the matching rows get returned in a new DataFrame, which is a subset of the original, containing only rows that matched the query. We can confirm that the result is still a DataFrame by checking the type of the variable:

In [ ]:
type(august2008)

This means that we can run all of the previously described DataFrame methods on the query results! For example, this can be useful for finding out how many rows matched the query:

In [ ]:
print(len(august2008.index)) # we can use the same code we saw earlier for counting rows, because august2008 is still a DataFrame

## Exercise 3

How many questions in 2013 had the tag "python"?

In [ ]:
python2013 = df.loc[______________] # first, fill in your query here
print(_____________) # then, fill in the code to count the number of rows

## 8. Modifying and adding rows
Queries aren't just useful for accessing data; we may sometimes also want to *modify* data that matches a query! One common reason to do this is for data cleaning. For example, some questions in our data set have the tag ".net" and others have the tag "net", but both of these actually refer to the same thing (the Microsoft dotNET framework). So when we clean the data, we might want to standardize all instances of ".net" to "net". This can be done with the following steps:

1. Write a query to find all instances of the thing you want to clean (in this case, questions with the tag ".net") (See Section 7)
2. Use row-and-column indexing to combine this query (which selects certain rows) with column selection (See Section 5)
3. Use Python assigment syntax (the "=" operator) to set the new value

Again, this is best understood through an example. The following code implements all the steps to standardize ".net" to "net":

In [ ]:
# Convert .net to net
df.loc[(df['tag'] == '.net'), 'tag'] = 'net'
# to confirm that it worked, count how many questions are now tagged "net" and how many are tagged ".net" (the latter should be zero)
print(len(df.loc[df['tag'] == 'net'].index))
print(len(df.loc[df['tag'] == '.net'].index))

In this example, we *replaced* values in an existing column. But there are other cases where instead, you might want to preserve the original data, and instead put the cleaned data in a new column. pandas lets us do this too! As an example, let's recall the missing values in the "accepted_id" column that we told you about earlier. Remember that the interpretation of these missing values is that the question did not receive an accepted answer. For some research questions, we might simply want to know *whether* a question received an accepted answer or not, without caring about the ID of the answer. But this does not mean we want to completely replace the "accepted_id" column, because for *other* research questions we might still care about the answer IDs! So instead, we might want to create a *new* column that contains the simplified information of whether a question received an accepted answer.

We will start by creating a new column, let's call it "has_answer". Initially, we'll create the column as a copy of "accepted_id"; we can then modify it (using code similar to what we did previously for "tag") without affecting the original column. To create a new column, we can simply use Python assigment syntax:

In [ ]:
df["has_answer"] = df["accepted_id"]
display(df)

We see that there is now a new column "has_answer" that is an exact copy of "accepted_id". Next we will modify it as follows: rows containing missing values will be set to 0 (representing "no answer") and the rest will be set to "1" (representing "has answer").

Syntax notes: to check if a value is missing, pandas provides the special comparison function `isna`. To negate a comparison (so we can say when a row does **not** contain a missing value), we'll use the the `~` operator.

In [ ]:
df.loc[~(pd.isna(df['has_answer'])), 'has_answer'] = 1
df.loc[(pd.isna(df['has_answer'])), 'has_answer'] = 0
display(df)

## 9. Fine-grained analysis using groupby
In section 6, we showed how to summarize data from an *entire* column, such as taking the min and max of numerical data. This is useful for initial exploration, but when tackling actual research questions, we often want to do more complicated operations involving interactions between multiple variables. For example, instead of just finding the earliest year in the data, we might be interested in finding the earliest year each tag first appeared. Using what we have learned so far, one way to do this would be to use queries: for each tag, you could write a query to select only the rows containing that tag, and call the `min` method each time. But this is extremely tedious, given that there are so many tags in the data! Thankfully, pandas offers a faster alternative: `groupby`.

The `groupby` method can be thought of a splitting a DataFrame based on some categorical variable. For example, if we use `groupby` on the "tag" column, we'll have one group containing rows tagged ".htaccess", another containing rows tagged "python", and so on, for every unique tag. Now, if we just run `groupby` on its own, we won't immediately see anything useful:

In [ ]:
df_grouped = df.groupby("tag")
display(df_grouped) # prints out some strange code that isn't super useful...

But the key difference happens when we call summarization methods on the grouped variable. Instead of running the summarization on the entire data, as happens normally, the summarization will be run separately on each group! Let's try using the `min` summarization method on "year" in the grouped data:

In [ ]:
display(df_grouped['year'].min()) # we can use the same indexing and summarization syntax as we did for regular DataFrames, but the operation now happens separately for each group!

As we can see, instead of a single minimum, we get multiple: one for each group! Specifically, we are seeing the earliest year each tag first appeared. We can immediately see how this might be useful for tracking trends; for example, questions tagged ".htaccess" started in 2010, while questions tagged ".htpasswd" didn't start until 2019. This might suggest, for example, that .htpasswd is a newer tag.

There are other kinds of summarization methods that can be useful when combined with `groupby`. Here are just a few examples:

In [ ]:
# The count() method counts the total number of unique items in a column.
# When combined with groupby, the counting will happen separately per group.
# Let's try using it to find out how many questions happened per year (by counting the number of unique question IDs per year group)
df_grouped = df.groupby("year")
display(df_grouped["id"].count())

In [ ]:
# The sum() method computes the sum of all items in a column
# When combined with groupby, sums are computed separately per group
# Let's try using it to find out how many questions were answered per year.
df_grouped = df.groupby("year")
display(df_grouped["has_answer"].sum())

Finally, note that it is possible to group on multiple columns at once! This will create a group for each unique pair of possible values in the two columns. For example, we might be interested not just in years, but in specific year-month combinations:

In [ ]:
df_grouped = df.groupby(["year", "month"])
display(df_grouped["id"].count())